## DEPENDECIES FOR ALL THE PROJECT 

In [90]:
# General purpose
import os
from dotenv import load_dotenv

# Extract (Web Scraping)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Transform & Load
import pandas as pd
import numpy as np
import sqlite3

# API
from flask import Flask, jsonify

# Visualize
from dash import Dash, dcc, html, Input, Output, State, ctx
import dash_bootstrap_components as dbc
import dash
from dash import dcc, html
import plotly.express as px



## STEP #1: EXTRACTION OF DATA FROM WEB WITH SELENIUM

In [91]:
# Setting up browser for Chrome
options = webdriver.ChromeOptions()

# Run Chrome in headless mode 
options.add_argument("--headless")  

# Launch Chrome using the options settings above
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)  



In [92]:
# Navigate browser to the bookscraping site
driver.get("https://books.toscrape.com/")


In [93]:
# Tell the browser to show us the title of the current page (to doubule check if loaded correctly)
print(driver.title)


All products | Books to Scrape - Sandbox


In [94]:
# Checking through the page and find all the books content (each book is inside an <article> tag with class 'product_pod')
books = driver.find_elements(By.CLASS_NAME, "product_pod")


In [95]:
# Let's count how many books we found on the page
print("Books found:", len(books))


Books found: 20


### SCRAPING PROCEDURE

In [96]:
# Let's grab the title of the first book on the page
first_title = books[0].find_element(By.TAG_NAME, "h3").find_element(By.TAG_NAME, "a").get_attribute("title")

# Show the title of the first book to confirm we got it right
print("First book title:", first_title)

First book title: A Light in the Attic


In [97]:
# Now let's get the price of the first book
first_price = books[0].find_element(By.CLASS_NAME, "price_color").text

# Show the price of the first book to confirm we got it right
print("First book price:", first_price)


First book price: £51.77


In [98]:
# Let's check if the first book is available (in stock or not)
first_availability = books[0].find_element(By.CLASS_NAME, "availability").text.strip()

# Show the availability of the first book to confirm we got it right
print("First book availability:", first_availability)


First book availability: In stock


In [99]:
# Let's get the rating of the first book (like 'Three', 'Four', etc.)
first_rating = books[0].find_element(By.CLASS_NAME, "star-rating").get_attribute("class").split()[-1]

# Show the rating of the first book to confirm we got it right
print("First book rating:", first_rating)

First book rating: Three


In [100]:
[
  {'title': '...', 'price': '...', 'availability': '...', 'rating': '...'},
  ...
]


[{'title': '...', 'price': '...', 'availability': '...', 'rating': '...'},
 Ellipsis]

In [101]:
# Create a list to hold all the book data
book_data = []


In [102]:
# Go through each book found on the page, one by one
for book in books:
	title = book.find_element(By.TAG_NAME, "h3").find_element(By.TAG_NAME, "a").get_attribute("title")
	price = book.find_element(By.CLASS_NAME, "price_color").text
	availability = book.find_element(By.CLASS_NAME, "availability").text.strip()
	rating = book.find_element(By.CLASS_NAME, "star-rating").get_attribute("class").split()[-1]
	book_data.append({
		'title': title,
		'price': price,
		'availability': availability,
		'rating': rating
	})


In [103]:
# Show the first 3 books collected to preview the results
print(book_data[:3])


[{'title': 'A Light in the Attic', 'price': '£51.77', 'availability': 'In stock', 'rating': 'Three'}, {'title': 'Tipping the Velvet', 'price': '£53.74', 'availability': 'In stock', 'rating': 'One'}, {'title': 'Soumission', 'price': '£50.10', 'availability': 'In stock', 'rating': 'One'}]


## STEP #2: TRANSFORM

In [104]:
# Convert the list of book data into a DataFrame for easier manipulation
df = pd.DataFrame(book_data)

In [105]:
# Visualizing the dataframe 
df.head()

,title,price,availability,rating
0,A Light in the Attic,£51.77,In stock,Three
1,Tipping the Velvet,£53.74,In stock,One
2,Soumission,£50.10,In stock,One
3,Sharp Objects,£47.82,In stock,Four
4,Sapiens: A Brief History of Humankind,£54.23,In stock,Five


In [106]:
# Removing the '£' symbol and convert the price column to float
df['price'] = df['price'].str.replace('£', '').astype(float)

In [107]:
# Verifying the changes without the '£' symbol
df.head()

,title,price,availability,rating
0,A Light in the Attic,51.77,In stock,Three
1,Tipping the Velvet,53.74,In stock,One
2,Soumission,50.10,In stock,One
3,Sharp Objects,47.82,In stock,Four
4,Sapiens: A Brief History of Humankind,54.23,In stock,Five


In [108]:
# Convert the rating words (e.g. 'Three') into numbers (e.g. 3.0)
rating_map = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
df['rating'] = df['rating'].map(rating_map).astype(float)


In [109]:
# Create a new column converting GBP to USD using a fixed exchange rate
exchange_rate = 1.28
df['price_usd'] = df['price'] * exchange_rate


In [110]:
# Show the updated DataFrame with the new USD prices
df.head()


,title,price,availability,rating,price_usd
0,A Light in the Attic,51.77,In stock,3.0,66.2656
1,Tipping the Velvet,53.74,In stock,1.0,68.7872
2,Soumission,50.10,In stock,1.0,64.1280
3,Sharp Objects,47.82,In stock,4.0,61.2096
4,Sapiens: A Brief History of Humankind,54.23,In stock,5.0,69.4144


In [111]:
# Modifying the column "price" for "price_gbp"
df.rename(columns={'price': 'price_gbp'}, inplace=True)

In [112]:
# Changing the column "availability" to "in_stock"
df.rename(columns={'availability': 'in_stock'}, inplace=True)

In [113]:
# Moving column "price_gbp" to the end of the DataFrame
df = df[['title', 'in_stock', 'rating', 'price_gbp', 'price_usd']]

In [114]:
# Verying the changes of column name "price" for "price_gbp", the position of the column, columns names and the data types
df.head()

,title,in_stock,rating,price_gbp,price_usd
0,A Light in the Attic,In stock,3.0,51.77,66.2656
1,Tipping the Velvet,In stock,1.0,53.74,68.7872
2,Soumission,In stock,1.0,50.10,64.1280
3,Sharp Objects,In stock,4.0,47.82,61.2096
4,Sapiens: A Brief History of Humankind,In stock,5.0,54.23,69.4144


## STEP 3: LOAD THE DATA INTO SQLITE & GET A CSV FILE 

In [115]:
# Load the DataFrame into a SQLite database
conn = sqlite3.connect('books.db')


In [116]:
# Write the DataFrame to a table named "books" in the SQLite database
df.to_sql('books', conn, if_exists='replace', index=False)  

20

In [117]:
# Export the cleaned DataFrame to a CSV file with condintional to avoid duplicateds files

if not os.path.exists("books_cleaned.csv"):
    df.to_csv("books_cleaned.csv", index=False)


## STEP 4: API BUILDING WITH THE DUE ENDPOINTS

In [118]:
# Get to the Python File "API_START" to run the API 


## ANALYSIS TO STABLISH THE ENDPOINTS INTO THE API

In [119]:
import sqlite3
conn = sqlite3.connect('books.db')
cursor = conn.cursor()
cursor.execute("PRAGMA table_info(books);")
print(cursor.fetchall())
conn.close()


[(0, 'title', 'TEXT', 0, None, 0), (1, 'in_stock', 'TEXT', 0, None, 0), (2, 'rating', 'REAL', 0, None, 0), (3, 'price_gbp', 'REAL', 0, None, 0), (4, 'price_usd', 'REAL', 0, None, 0)]


## DASHBOARD BUILDING USING DASH PLOTY 

In [121]:
app = dash.Dash(__name__)
server = app.server  # Para producción si se necesita

app.layout = html.Div(style={'fontFamily': 'Arial', 'textAlign': 'center'}, children=[
    html.H2("📚 Books Dashboard", style={'marginBottom': '40px'}),

    html.Div(style={'width': '50%', 'margin': '0 auto'}, children=[
        html.Label("⭐ Select Rating:"),
        dcc.Dropdown(
            id='rating_dropdown',
            options=[{'label': f"{i} Star{'s' if i > 1 else ''}", 'value': i} for i in range(1, 6)],
            placeholder="Choose a rating...",
            style={'marginBottom': '20px'}
        ),

        html.Label("💷 Filter by Price Range (GBP):"),
        dcc.Input(id='min_price', type='number', placeholder='Min price', style={'marginRight': '10px'}),
        dcc.Input(id='max_price', type='number', placeholder='Max price', style={'marginBottom': '20px'}),

        html.Br(),

        html.Label("🔍 Search Title by Keyword:"),
        dcc.Input(id='keyword_input', type='text', placeholder='Enter keyword...', style={'marginBottom': '20px'}),

        html.Br(),

        dcc.Checklist(
            id='in_stock_checkbox',
            options=[{'label': '📦 Only show In Stock books', 'value': 'in_stock'}],
            style={'marginBottom': '20px', 'marginTop': '10px'}
        ),

        html.Button('Submit', id='submit_button', n_clicks=0, style={'marginBottom': '30px'}),
    ]),

    html.Div(id='results_output')
])

@app.callback(
    Output('results_output', 'children'),
    Input('submit_button', 'n_clicks'),
    State('min_price', 'value'),
    State('max_price', 'value'),
    State('keyword_input', 'value'),
    State('in_stock_checkbox', 'value'),
    State('rating_dropdown', 'value')
)
def update_output(n_clicks, min_price, max_price, keyword, in_stock, rating_value):
    filters = []

    if min_price is not None and max_price is not None:
        try:
            url = f"http://127.0.0.1:5000/books/price/{min_price}/{max_price}"
            response = requests.get(url)
            if response.status_code == 200:
                filters = response.json()
        except:
            return "❌ Error fetching price filtered data."
    else:
        response = requests.get("http://127.0.0.1:5000/books")
        if response.status_code == 200:
            filters = response.json()

    if keyword:
        filters = [book for book in filters if keyword.lower() in book['title'].lower()]

    if in_stock and 'in_stock' in in_stock:
        filters = [book for book in filters if book['in_stock'].lower() == 'in stock']

    if rating_value:
        filters = [book for book in filters if int(float(book.get('rating', 0))) == rating_value]

    if not filters:
        return html.Div("🔍 No books found with the selected filters.", style={'color': 'gray', 'marginTop': '30px'})

    return html.Div([
        html.H4("📘 Matching Books", style={'color': '#007BFF'}),
        html.Hr(),
        *[html.Div([
            html.H5(f"📖 Title: {book['title']}", style={'fontWeight': 'bold'}),
            html.P(f"💷 Price (GBP): £{book['price_gbp']}"),
            html.P(f"💲 Price (USD): ${book['price_usd']}"),
            html.P(f"⭐ Rating: {book['rating']}"),
            html.P(f"📦 Availability: {book['in_stock']}"),
            html.Hr()
        ]) for book in filters]
    ])

if __name__ == '__main__':
    app.run_server(debug=True, port=8050)

